## Reading jsons

In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

time = pd.read_json('points_1000000.1_time.json', typ='series', orient='index')
time_compact = pd.read_json('points_1000000.1_time_compact.json', typ='series', orient='index')

time2 = pd.read_json('points_1500000.1_time.json', typ='series', orient='index')
time_compact2 = pd.read_json('points_1500000.1_time_compact.json', typ='series', orient='index')

#pd.concat([time, time2], axis=1)

import os,json

path_to_json = '/'



path_to_json = '.'
times_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('_time.json')]
compact_time_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('_time_compact.json')]


#data_time = pd.DataFrame(columns=['time_triangulation_generation','time_to_label_max_edges', 'time_to_label_frontier_edges', 'time_to_label_seed_edges', 'time_to_traversal_and_repair', 'time_to_traversal', 'time_to_repair', 'polygons', 'n_frontier_edges', 'n_barrier_edge_tips', 'n_half_edges', 'n_faces', 'n_vertices'])
data_to_append = []

for js in times_files:
    data_to_append.append(pd.read_json(js, typ='series', orient='index'))

data_time = pd.concat(data_to_append, axis=1)
data_time = data_time.transpose()
data_time = data_time.set_index('n_vertices')

compact_data_to_append = []

for js in compact_time_files:
    compact_data_to_append.append(pd.read_json(js, typ='series', orient='index'))

data_time_compact = pd.concat(compact_data_to_append, axis=1)
data_time_compact = data_time_compact.transpose()
data_time_compact = data_time_compact.set_index('n_vertices')


# Comp time vs compact time

,time_triangulation_generation,time_to_label_max_edges,time_to_label_frontier_edges,time_to_label_seed_edges,time_to_traversal_and_repair,time_to_traversal,time_to_repair,polygons,n_frontier_edges,n_barrier_edge_tips,n_half_edges,n_faces
n_vertices,,,,,,,,,,,,
2500000.0,2842.24,6081.48,1727.910,2172.560,106932.0,2813.78,104119.0,804912.0,3305193.0,49236.0,14992112.0,4996057.0
4500000.0,5197.81,10055.30,3179.580,3877.430,346114.0,4955.31,341158.0,1450420.0,5950973.0,89788.0,26989490.0,8994746.0
3500000.0,4123.37,8142.85,2409.480,3036.200,209011.0,3946.82,205064.0,1127561.0,4627989.0,69782.0,20990756.0,6995379.0
4000000.0,4571.31,9511.27,2770.480,3536.960,277824.0,4459.29,273365.0,1288772.0,5289255.0,79414.0,23990090.0,7995046.0
3000000.0,3399.05,6974.02,2106.110,2593.940,150392.0,3282.84,147109.0,966410.0,3966812.0,59482.0,17991414.0,5995708.0
1000000.0,1254.54,2302.05,724.717,918.507,11816.1,1158.87,10657.3,322457.0,1322592.0,19649.0,5994992.0,1997497.0
1500000.0,1898.17,3781.23,1211.200,1483.230,32533.2,1671.17,30862.0,484160.0,1984344.0,30105.0,8993994.0,2996998.0
2000000.0,2389.16,4694.14,1474.450,1727.360,67135.6,2279.24,64856.3,644402.0,2644629.0,39633.0,11992976.0,3996489.0
